In [1]:
## PREPROCESS
# chia dataset ra
# thêm negative là dự đoán của bge-m3

## HYBRID SEARCH
# dùng BM25 để lấy top 20
# tạo Dataset class phù hợp vói triplet loss
# fine tune bi-encoder PhoBERT triplet loss với LoRA và lấy top 10
# hợp nhất và lấy top 25 bằng weighted sum score

## RERANK
# tạo Dataset class phù hợp vói binary classification
# fine tune cross encoder PhoBERT với binary classification

# EVAL
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


# PREPROCESS

In [2]:
import pandas as pd
corpus = pd.read_csv("/kaggle/input/retrieval/corpus.csv")
train = pd.read_csv("/kaggle/input/retrieval/train.csv")

## DATASET

In [3]:
import pandas as pd
import re

def prepare_data(train_df, corpus_df, sample_size=500, max_corpus_size=10000, output_prefix="hybrid_search", random_state=42):
    import re

    # Bước 1: Sample dữ liệu huấn luyện
    train_sampled = train_df.sample(n=sample_size, random_state=random_state)

    # Bước 2: Làm sạch cột cid
    def robust_clean_cid_string(s):
        if not isinstance(s, str):
            return []
        s = s.replace('[', '').replace(']', '').replace("'", '').replace('"', '').strip()
        s = re.sub(r'\s+', ',', s)
        try:
            return [int(x) for x in s.split(',') if x.strip().isdigit()]
        except Exception as e:
            print(f"⚠️ Lỗi parse cid: {s} → {e}")
            return []

    train_sampled['cid'] = train_sampled['cid'].apply(robust_clean_cid_string)

    # Bước 3: Lọc các sample có cid hợp lệ
    available_cids = set(corpus_df['cid'].unique())
    train_sampled = train_sampled[train_sampled['cid'].apply(lambda cids: all(cid in available_cids for cid in cids))]

    # Bước 4: Tạo positive_context
    all_cids = set(cid for sublist in train_sampled['cid'] for cid in sublist)
    cid_to_text = corpus_df.set_index("cid")['text'].to_dict()
    train_sampled['positive_context'] = train_sampled['cid'].apply(
        lambda cids: "\n".join([cid_to_text.get(cid, "") for cid in cids if cid in cid_to_text])
    )

    # Bước 5: Tạo corpus_sampled đủ max_corpus_size
    corpus_positive = corpus_df[corpus_df['cid'].isin(all_cids)].copy()
    num_needed = max_corpus_size - len(corpus_positive)

    if num_needed > 0:
        # Lấy thêm các sample ngẫu nhiên không trùng
        remaining_corpus = corpus_df[~corpus_df['cid'].isin(all_cids)]
        corpus_additional = remaining_corpus.sample(n=num_needed, random_state=random_state)
        corpus_sampled = pd.concat([corpus_positive, corpus_additional], ignore_index=True)
    else:
        # Nếu đã đủ hoặc dư thì chỉ lấy ngẫu nhiên đúng max_corpus_size dòng
        corpus_sampled = corpus_positive.sample(n=max_corpus_size, random_state=random_state)

    # Bước 6: Clean và lưu
    train_sampled = train_sampled.dropna(subset=['question', 'positive_context'])
    train_sampled.to_csv(f"train_{output_prefix}.csv", index=False)
    corpus_sampled.to_csv(f"corpus_{output_prefix}.csv", index=False)

    print(f"✅ Saved: {len(train_sampled)} training rows and {len(corpus_sampled)} corpus rows.")



prepare_data(train, corpus, sample_size=5000, max_corpus_size=10000, output_prefix="hybrid_search", random_state=123)
prepare_data(train, corpus, sample_size=5000, max_corpus_size=10000, output_prefix="rerank", random_state=456)

✅ Saved: 4975 training rows and 10000 corpus rows.
✅ Saved: 4974 training rows and 10000 corpus rows.


In [4]:
import pandas as pd
import ast

def check_cid_consistency(train_path, corpus_path):
    train_df = pd.read_csv(train_path)
    corpus_df = pd.read_csv(corpus_path)

    # Chuẩn hóa cột cid từ string → list[int]
    def parse_cid(x):
        if isinstance(x, list):
            return x
        try:
            return ast.literal_eval(x)
        except:
            return []

    train_df['cid'] = train_df['cid'].apply(parse_cid)

    # Tập hợp tất cả cid xuất hiện trong train
    train_cids = set(cid for sublist in train_df['cid'] for cid in sublist)

    # Tập hợp toàn bộ cid trong corpus
    corpus_cids = set(corpus_df['cid'].unique())

    # So sánh
    missing_cids = train_cids - corpus_cids

    if not missing_cids:
        print("✅ Tất cả CID trong train đều có trong corpus.")
    else:
        print(f"❌ Có {len(missing_cids)} CID bị thiếu trong corpus:")
    return missing_cids
check_cid_consistency(train_path="train_rerank.csv", corpus_path="corpus_rerank.csv")
check_cid_consistency(train_path="train_hybrid_search.csv", corpus_path="corpus_hybrid_search.csv")

✅ Tất cả CID trong train đều có trong corpus.
✅ Tất cả CID trong train đều có trong corpus.


set()

## thêm hard negative bằng dự đoán của bge-m3

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def generate_hard_negatives_with_bge(
    train_file: str,
    corpus_file: str,
    output_file: str = "train_neg_hybrid_search.csv",
    model_name: str = "BAAI/bge-m3",
    batch_size: int = 64,
    max_length: int = 256
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    # Load data
    train_sampled = pd.read_csv(train_file)
    corpus_sampled = pd.read_csv(corpus_file)

    # Lấy text corpus và map cid
    cid_to_text = corpus_sampled.set_index("cid")['text'].to_dict()
    all_corpus_texts = list(cid_to_text.values())
    all_corpus_cids = list(cid_to_text.keys())

    # Hàm encode
    def get_embedding(texts):
        inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            return outputs.last_hidden_state[:, 0].cpu().numpy()  # [CLS]

    # Embed corpus
    print("🔄 Encoding corpus...")
    corpus_embeddings = []
    for i in tqdm(range(0, len(all_corpus_texts), batch_size)):
        batch_texts = all_corpus_texts[i:i+batch_size]
        batch_emb = get_embedding(batch_texts)
        corpus_embeddings.append(batch_emb)
    corpus_embeddings = np.vstack(corpus_embeddings)

    # Tạo hard negatives
    print("🔍 Generating hard negatives...")
    negatives = []

    for idx, row in tqdm(train_sampled.iterrows(), total=len(train_sampled)):
        question = row['question']
        pos_cids = set(row['cid']) if isinstance(row['cid'], list) else set()

        q_emb = get_embedding([question])
        sims = cosine_similarity(q_emb, corpus_embeddings)[0]
        top_indices = sims.argsort()[::-1]

        selected_neg = None
        for top_idx in top_indices:
            neg_cid = all_corpus_cids[top_idx]
            if neg_cid not in pos_cids:
                selected_neg = all_corpus_texts[top_idx]
                break

        negatives.append(selected_neg or "no hard negative found")

    # Gán và lưu
    train_sampled['negative'] = negatives
    train_sampled = train_sampled.dropna(subset=['question', 'positive_context', 'negative'])
    train_sampled = train_sampled.drop(columns=['positive_context'])
    train_sampled.to_csv(output_file, index=False)

    print(f"✅ Đã tạo negative và lưu: {output_file} ({len(train_sampled)} samples)")
    return train_sampled


In [6]:
generate_hard_negatives_with_bge(
    train_file="train_hybrid_search.csv",
    corpus_file="corpus_hybrid_search.csv",
    output_file="train_neg_hybrid_search.csv"
)

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

100%|██████████| 157/157 [04:06<00:00,  1.57s/it]


🔍 Generating hard negatives...


100%|██████████| 4975/4975 [05:32<00:00, 14.95it/s]


✅ Đã tạo negative và lưu: train_neg_hybrid_search.csv (4975 samples)


,question,context,cid,qid,negative
0,Người vi phạm quy định về phòng cháy chữa cháy...,"['Tội vi phạm quy định về phòng cháy, chữa chá...",[61969],124743,Khoản 3. Biện pháp khắc phục hậu quả:\na) Buộc...
1,Chi nhánh của tổ chức hòa giải thương mại nước...,"['Quyền và nghĩa vụ của chi nhánh, văn phòng đ...",[31484],24462,"Quyền và nghĩa vụ của chi nhánh, văn phòng đại..."
2,Tài xế dương tính ma túy điều khiển phương tiệ...,"['Điều 82. Tạm giữ phương tiện, giấy tờ có liê...",[61478],81790,"""Điều 82. Tạm giữ phương tiện, giấy tờ có liên..."
3,Ủy ban nhân dân có thẩm quyền hoạt động về thẩ...,['Thẩm quyền quản lý nhà nước về thẩm định giá...,[185298],110549,Thẩm quyền quản lý nhà nước về thẩm định giá\n...
4,Người bán hàng chèo kéo khách ở khu vực lễ hội...,['Vi phạm quy định về tổ chức lễ hội\n1. Phạt ...,[41036],142434,Vi phạm quy định về tổ chức lễ hội\n1. Phạt cả...
...,...,...,...,...,...
4970,Doanh nghiệp đăng ký phân loại doanh nghiệp ch...,['Đăng ký phân loại doanh nghiệp\nDoanh nghiệp...,[73303],10608,Đăng ký phân loại doanh nghiệp\nDoanh nghiệp đ...
4971,Doanh nghiệp FDI có trách nhiệm như thế nào kh...,['Trách nhiệm của doanh nghiệp có vốn đầu tư t...,[6823],69360,Trách nhiệm của doanh nghiệp có vốn đầu tư trự...
4972,Người học ngành quản lý đô thị trình độ trung ...,"['Khả năng học tập, nâng cao trình độ\n- Khối ...",[62492],63170,1. Đào tạo trình độ trung cấp yêu cầu người họ...
4973,Mẫu quyết định cấp thẻ thanh tra chuyên ngành ...,"['Hồ sơ, thủ tục và thẩm quyền cấp thẻ\n1. Hồ ...",[80451],47151,"Hồ sơ, thủ tục và thẩm quyền cấp thẻ\n...\n3. ..."


In [7]:
generate_hard_negatives_with_bge(
    train_file="train_rerank.csv",
    corpus_file="corpus_rerank.csv",
    output_file="train_neg_rerank.csv"
)

🔄 Encoding corpus...


100%|██████████| 157/157 [04:05<00:00,  1.56s/it]


🔍 Generating hard negatives...


100%|██████████| 4974/4974 [05:27<00:00, 15.17it/s]


✅ Đã tạo negative và lưu: train_neg_rerank.csv (4974 samples)


,question,context,cid,qid,negative
0,Bình đẳng giới trong gia đình được thể hiện qu...,"['Bình đẳng giới trong gia đình\n1. Vợ, chồng ...",[53911],121967,"Bình đẳng giới trong gia đình\n1. Vợ, chồng bì..."
1,Khiến người đang ở trong tình trạng quẫn bách ...,['Tội cưỡng dâm\n1. Người nào dùng mọi thủ đoạ...,"[67438, 67438]",122529,"""Điều 257. Tội cưỡng bức người khác sử dụng tr..."
2,Hiệp hội công chứng viên Việt Nam có những nhi...,['Nhiệm vụ và quyền hạn của Hiệp hội công chứn...,[36322],104501,Nhiệm vụ và quyền hạn của Hiệp hội công chứng ...
3,Hồ sơ đề nghị điều chỉnh nội dung giấy phép ho...,"['Hồ sơ đề nghị gia hạn, điều chỉnh nội dung g...",[68054],18818,"Hồ sơ đề nghị gia hạn, điều chỉnh nội dung giấ..."
4,Thời hiệu xử phạt vi phạm hành chính đối với c...,['Thời hiệu xử lý vi phạm hành chính\n1. Thời ...,[61543],46122,"""Điều 3. Thời hiệu xử phạt vi phạm hành chính\..."
...,...,...,...,...,...
4969,Viên chức bị phân loại đánh giá ở mức độ không...,"['Sử dụng kết quả đánh giá công chức, viên chứ...",[61585],115752,"Sử dụng kết quả đánh giá công chức, viên chức ..."
4970,Điều kiện để được tổ chức cuộc thi hoa hậu hiệ...,"['Điều kiện, thủ tục tổ chức cuộc thi người đẹ...",[79367],119640,"Điều kiện, thủ tục tổ chức cuộc thi, liên hoan..."
4971,Thời gian nhận Phiếu đăng ký dự tuyển công chứ...,['Thông báo tuyển dụng và tiếp nhận Phiếu đăng...,[117225],81889,Thông báo tuyển dụng và tiếp nhận Phiếu đăng k...
4972,Ai có thẩm quyền giải quyết khiếu nại quyết đị...,['Thẩm quyền giải quyết khiếu nại quyết định k...,[151701],80526,Thẩm quyền giải quyết khiếu nại quyết định kỷ ...


In [8]:

# Load file
df = pd.read_csv("train_neg_rerank.csv")  # hoặc tên file bạn đang dùng

# Tạo 2 bản ghi cho mỗi dòng: 1 positive và 1 negative
pos_rows = df[["question", "context"]].copy()
pos_rows["label"] = 1
pos_rows = pos_rows.rename(columns={"context": "context"})

neg_rows = df[["question", "negative"]].copy()
neg_rows["label"] = 0
neg_rows = neg_rows.rename(columns={"negative": "context"})

# Gộp lại
rerank_df = pd.concat([pos_rows, neg_rows], ignore_index=True)

# Shuffle để tránh model bias
rerank_df = rerank_df.sample(frac=1.0, random_state=42).reset_index(drop=True)

# Lưu
rerank_df.to_csv("rerank_train.csv", index=False)
print("✅ Saved rerank_train.csv with shape:", rerank_df.shape)


✅ Saved rerank_train.csv with shape: (9948, 3)


# HYBRID SEARCH

## BM25 để lấy top 20

In [9]:
!pip install rank_bm25

In [10]:
import pandas as pd
from rank_bm25 import BM25Okapi

def retrieve_bm25(query, corpus_path, top_k=30):
    corpus = pd.read_csv(corpus_path)
    bm25 = BM25Okapi(corpus["text"].astype(str).str.split())
    
    scores = bm25.get_scores(query.split())
    corpus["score"] = scores

    top_docs = corpus.sort_values(by="score", ascending=False).head(top_k).reset_index(drop=True)
    return top_docs[["cid", "text", "score"]]  # hoặc thêm các cột bạn cần


## Bi-encoder để lấy top 10

In [11]:
from torch.utils.data import Dataset
import pandas as pd

class LegalBiEncoderDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row['question']
        pos = row['context']
        neg = row['negative']

        anchor = self.tokenizer(question, padding='max_length', truncation=True,
                                max_length=self.max_length, return_tensors="pt")
        positive = self.tokenizer(pos, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")
        negative = self.tokenizer(neg, padding='max_length', truncation=True,
                                  max_length=self.max_length, return_tensors="pt")

        return {
            "anchor_input_ids": anchor["input_ids"].squeeze(),
            "anchor_attention_mask": anchor["attention_mask"].squeeze(),
            "positive_input_ids": positive["input_ids"].squeeze(),
            "positive_attention_mask": positive["attention_mask"].squeeze(),
            "negative_input_ids": negative["input_ids"].squeeze(),
            "negative_attention_mask": negative["attention_mask"].squeeze()
        }


In [12]:
import torch
import torch.nn as nn
from transformers import AutoModel
from peft import get_peft_model, LoraConfig, TaskType

class BiEncoderPhoBERT(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        base_model = AutoModel.from_pretrained(model_name)

        # LoRA config
        peft_config = LoraConfig(
            task_type=TaskType.FEATURE_EXTRACTION,
            r=8,
            lora_alpha=16,
            lora_dropout=0.1,
            target_modules=["encoder.layer.11.attention.output.dense"],  # hoặc mở rộng thêm
            bias="none"
        )
        self.encoder = get_peft_model(base_model, peft_config)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = output.last_hidden_state[:, 0]  # [CLS] vector
        return cls_output


In [13]:
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "vinai/phobert-base"
max_length = 256
epochs = 15
batch_size = 32
lr = 2e-5


tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
dataset = LegalBiEncoderDataset("/kaggle/working/train_neg_hybrid_search.csv", tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = BiEncoderPhoBERT(model_name).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = nn.TripletMarginLoss(margin=0.3)

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(dataloader):
        anchor = model(batch['anchor_input_ids'].to(device), batch['anchor_attention_mask'].to(device))
        pos = model(batch['positive_input_ids'].to(device), batch['positive_attention_mask'].to(device))
        neg = model(batch['negative_input_ids'].to(device), batch['negative_attention_mask'].to(device))

        loss = loss_fn(anchor, pos, neg)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}: Loss = {total_loss:.4f}")


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

...

Epoch 13: Loss = 1.2465


100%|██████████| 156/156 [02:19<00:00,  1.12it/s]


Epoch 14: Loss = 1.1670


100%|██████████| 156/156 [02:19<00:00,  1.12it/s]

Epoch 15: Loss = 1.4038


In [14]:

import os
output_dir = "checkpoints/bi_encoder_phobert"
os.makedirs(output_dir, exist_ok=True)

model.encoder.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Đã lưu checkpoint tại {output_dir}")

✅ Đã lưu checkpoint tại checkpoints/bi_encoder_phobert


In [15]:
import torch
from transformers import AutoTokenizer, AutoModel
from peft import PeftModel, PeftConfig
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load corpus (chứa cid, text)
corpus_df = pd.read_csv("/kaggle/working/corpus_rerank.csv")
corpus_texts = corpus_df["text"].tolist()
corpus_cids = corpus_df["cid"].tolist()

# Load model + tokenizer
def load_dense_model(checkpoint_path="checkpoints/bi_encoder_phobert"):
    config = PeftConfig.from_pretrained(checkpoint_path)
    base = AutoModel.from_pretrained(config.base_model_name_or_path)
    model = PeftModel.from_pretrained(base, checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, use_fast=False)
    return model.eval(), tokenizer

# Encode corpus (chạy một lần hoặc lưu sẵn)
def encode_corpus(model, tokenizer, device, max_len=256):
    embeddings = []
    model.eval()
    model.to(device)

    with torch.no_grad():
        for text in corpus_texts:
            inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_len, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            output = model(inputs["input_ids"], inputs["attention_mask"])
            cls_output = output.last_hidden_state[:, 0]  # [CLS] token
            emb = cls_output.cpu().numpy()[0]
            embeddings.append(emb)

    return torch.tensor(embeddings)


# Build index
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, tokenizer = load_dense_model()
corpus_embeddings = encode_corpus(model, tokenizer, device)

# Define dense_func
def dense_func(query, top_k=50):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(query, truncation=True, padding='max_length', max_length=256, return_tensors="pt").to(device)
        output = model(inputs["input_ids"], inputs["attention_mask"])
        cls_output = output.last_hidden_state[:, 0]  # lấy vector [CLS]
        query_vec = cls_output.cpu().numpy()

    sims = cosine_similarity(query_vec, corpus_embeddings.numpy())[0]
    top_indices = sims.argsort()[::-1][:top_k]

    results = []
    for idx in top_indices:
        results.append({
            "cid": corpus_cids[idx],
            "text": corpus_texts[idx],
            "score": float(sims[idx]) # chỉ để debug thôi, sau cần xóa
        })

    return results



<ipython-input-15-24b477da371a>:35: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(embeddings)


In [16]:
import numpy as np

def normalize(scores):
    scores = np.array(scores, dtype=np.float32)
    return (scores - scores.min()) / (scores.max() - scores.min() + 1e-6)

def hybrid_search2(query, bm25_func, dense_func, corpus, weight_bm25=0.5, weight_dense=0.5, top_k=40):
    # Lấy kết quả từ cả hai phía
    bm25_res = bm25_func(query, corpus)
    dense_res = dense_func(query)

    # Chuẩn hóa điểm
    bm25_scores = [item["score"] for item in bm25_res.to_dict("records")]
    dense_scores = [item["score"] for item in dense_res]

    bm25_scores_norm = normalize(bm25_scores)
    dense_scores_norm = normalize(dense_scores)

    # Gán lại điểm chuẩn hóa vào bản ghi
    bm25_list = bm25_res.to_dict("records")
    for i, item in enumerate(bm25_list):
        item["score"] = bm25_scores_norm[i]

    for i, item in enumerate(dense_res):
        item["score"] = dense_scores_norm[i]

    # Tạo map từ cid
    bm25_map = {item["cid"]: item for item in bm25_list}
    dense_map = {item["cid"]: item for item in dense_res}

    # Hợp nhất + tính score kết hợp
    all_cids = set(bm25_map) | set(dense_map)
    merged = []
    for cid in all_cids:
        text = bm25_map.get(cid, dense_map.get(cid))["text"]
        bm25_score = bm25_map.get(cid, {}).get("score", 0.0)
        dense_score = dense_map.get(cid, {}).get("score", 0.0)
        final_score = weight_bm25 * bm25_score + weight_dense * dense_score
        merged.append({
            "cid": cid,
            "text": text,
            "score": final_score
        })

    # Sắp xếp theo điểm kết hợp
    return sorted(merged, key=lambda x: x["score"], reverse=True)[:top_k]


### TESTING

In [17]:
# testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
results = dense_func(query)
results

[{'cid': 502715,
  'text': "Phước Long 09° 19' 08'' 105° 30' 18'' 09° 20' 46'' 105° 30' 28'' C-48-68-A-c kênh Ranh Dân Quân TV xã Vĩnh Thanh H. Phước Long 09° 20' 43'' 105° 26' 48'' 09° 19' 47'' 105° 30' 50'' C-48-67-B-d, C-48-68-A-c kênh Tiệm May TV xã Vĩnh Thanh H. Phước Long 09° 20' 34'' 105° 28' 19'' 09° 22' 30'' 105° 28' 33'' C-48-67-B-d kênh Tư Quán TV xã Vĩnh Thanh H. Phước Long 09° 23' 24'' 105° 29' 40'' 09° 21' 10'' 105° 29' 10'' C-48-67-B-b, C-48-67-B-d kênh Tường Thắng TV xã Vĩnh Thanh H. Phước Long 09° 20' 45'' 105° 30' 59'' 09° 19' 50'' 105° 31' 00'' C-48-68-A-c kênh Trưởng Tòa TV xã Vĩnh Thanh H. Phước Long 09° 21' 30'' 105° 31' 11'' 09° 20' 45'' 105° 30' 59'' C-48-68-A-c kênh Vĩnh Mỹ -Phước Long TV xã Vĩnh Thanh H. Phước Long 09° 16' 17'' 105° 35' 17'' 09° 26' 17'' 105° 27' 33'' C-48-67-B-b, C-48-68-A-a, C-48-68-A-c kênh Vĩnh Phong TV xã Vĩnh Thanh H. Phước Long 09° 24' 28'' 105° 25' 37'' 09° 15' 39'' 105° 32' 48'' C-48-67-B-d kênh Vĩnh Phong 6 TV xã Vĩnh Thanh H. Phước 

In [18]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
corpus = "/kaggle/working/corpus_hybrid_search.csv"
bm25_results = retrieve_bm25(query,corpus)
bm25_results

,cid,text,score
0,89598,"Nhiệm vụ, quyền hạn của Tổng Giám đốc.\n1. Tổ ...",42.396401
1,638236,"Khoản 4. VINATEX được quyền quyết định thang, ...",42.014820
2,563245,Khoản 3. Cột 10 ghi hệ số lương mới được chuyể...,39.996683
3,64854,Quyền lợi và trách nhiệm của thành viên Hội đồ...,39.075514
4,610353,b) Xếp lương ngạch chuyên viên chính hoặc ngạc...,38.408439
5,9459,"Kinh phí thực hiện điều chỉnh lương hưu, trợ c...",35.921831
6,451591,b) Điều kiện và tiêu chuẩn xếp hạng công ty tạ...,35.567081
7,63324,Phạm vi và đối tượng\n...\n2. Đối tượng không ...,35.394532
8,449726,Khoản 3. Đối tượng quy định tại Khoản 1 và Kho...,34.875009
9,529780,Điều 4. Xếp lương và phụ cấp lương\n1. Đối tượ...,34.326356


In [19]:
## testing
query = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
hybrid_searching = hybrid_search2(query, retrieve_bm25, dense_func, corpus )
hybrid_searching

[{'cid': 89598,
  'text': 'Nhiệm vụ, quyền hạn của Tổng Giám đốc.\n1. Tổ chức triển khai thực hiện các nghị quyết, quyết định của Hội đồng quản trị.\n2. Tổ chức điều hành các hoạt động nghiệp vụ của Ngân hàng Chính sách xã hội.\n3. Cùng Chủ tịch Hội đồng quản trị ký nhận vốn và các nguồn lực khác do Nhà nước giao.\n4. Ban hành các văn bản hướng dẫn nghiệp vụ.\n5. Ký các văn bản, thoả ước, hợp đồng, chứng thư của Ngân hàng Chính sách xã hội trong công tác đối nội, đối ngoại sau khi có ý kiến chuẩn y của Hội đồng quản trị.\n6. Tổ chức đào tạo tay nghề, phổ biến các chủ trương, chính sách, quy chế về nghiệp vụ.\n7. Trình Hội đồng quản trị:\na) Các công việc quy định tại khoản 2 Điều 23 của Điều lệ này;\nb) Sửa đổi, bổ sung Điều lệ về tổ chức và hoạt động của Ngân hàng Chính sách xã hội;\nc) Mở, thành lập, sáp nhập, chia tách, chấm dứt hoạt động các Chi nhánh và các tổ chức khác trong hệ thống Ngân hàng Chính sách xã hội.\n8. Ban hành Quy chế điều hành tại Hội sở chính, Chi nhánh, Phòng gi

# RERANK


In [20]:
from torch.utils.data import Dataset
import pandas as pd

class RerankDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=256):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        question = row["question"]
        context = row["context"]
        label = row["label"]  # 1 = relevant, 0 = not relevant

        encoded = self.tokenizer(
            question,
            context,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": int(label)
        }


In [21]:

os.environ["WANDB_DISABLED"] = "true"
import warnings
warnings.filterwarnings("ignore", message="Be aware, overflowing tokens.*")


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

model_name = "vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Load dataset
df = pd.read_csv("rerank_train.csv")  # Cột: question, context, label
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_csv("rerank_train_split.csv", index=False)
val_df.to_csv("rerank_val_split.csv", index=False)

train_dataset = RerankDataset("rerank_train_split.csv", tokenizer)
val_dataset = RerankDataset("rerank_val_split.csv", tokenizer)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training args
training_args = TrainingArguments(
    output_dir="./rerank_ckpt",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=7,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2
)

# Metric
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

# Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


# Train
trainer.train()


In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

def rerank_with_cross_encoder(question, contexts, model_path, model_name="vinai/phobert-base", top_k=10, device=None, max_length=256):
    import torch
    from transformers import AutoTokenizer, AutoModelForSequenceClassification

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model & tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device).eval()

    # ✅ Làm sạch context: loại None, cắt dài
    contexts = [c if isinstance(c, str) else "" for c in contexts]
    contexts = [c[:2048] for c in contexts]  # ngăn tokenizer tạo >512 tokens

    try:
        inputs = tokenizer(
            [question] * len(contexts),
            contexts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)
    except Exception as e:
        print("❌ Tokenizer error:", e)
        print("👉 Context length:", [len(c) for c in contexts])
        raise

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[:, 1]

    results = list(zip(contexts, probs.cpu().tolist()))
    reranked = sorted(results, key=lambda x: x[1], reverse=True)

    return reranked[:top_k]



In [24]:
best_ckpt_path = trainer.state.best_model_checkpoint
print("✅ Best checkpoint path:", best_ckpt_path)


✅ Best checkpoint path: ./rerank_ckpt/checkpoint-1120


In [ ]:
# Một example
question = "Phó Tổng Giám đốc Ngân hàng Chính sách xã hội được xếp lương theo bảng lương như thế nào?"
hybrid_searching = hybrid_search2(question, retrieve_bm25, dense_func, corpus )
contexts = [item['text'] for item in hybrid_searching]

reranked = rerank_with_cross_encoder(
    question=question,
    contexts=contexts,
    model_path=best_ckpt_path, 
    model_name=model_name,
    top_k=10,
    device="cpu"
)

# In kết quả
for i, (ctx, score) in enumerate(reranked, 1):
    print(f"#{i} - score: {score:.4f}")
    print(ctx)
    print("-" * 50)


AttributeError: 'SequenceClassifierOutput' object has no attribute 'last_hidden_state'

In [ ]:
import numpy as np

def evaluate_cross_lingual_rag(eval_data, corpus, retrieve_bm25, dense_func,
                                model_path, model_name, top_k=10, device="cpu"):
    recall_scores = []
    mrr_scores = []

    for example in eval_data:
        question = example["question"]
        relevant_cids = set(example["relevant_cids"])

        # Hybrid search: BM25 + dense
        hybrid_results = hybrid_search2(question, retrieve_bm25, dense_func, corpus)

        # Extract texts and cids
        contexts = [item['text'] for item in hybrid_results]
        cids = [item['cid'] for item in hybrid_results]

        # Rerank
        reranked = rerank_with_cross_encoder(
            question=question,
            contexts=contexts,
            model_path=model_path,
            model_name=model_name,
            top_k=top_k,
            device=device
        )

        # Map text → cid
        reranked_cids = []
        for ctx, _ in reranked:
            for item in hybrid_results:
                if item['text'].strip() == ctx.strip():
                    reranked_cids.append(item['cid'])
                    break

        # Recall@k
        hit = any(cid in relevant_cids for cid in reranked_cids)
        recall_scores.append(1.0 if hit else 0.0)

        # MRR@k
        rr = 0.0
        for rank, cid in enumerate(reranked_cids, 1):
            if cid in relevant_cids:
                rr = 1.0 / rank
                break
        mrr_scores.append(rr)

    avg_recall = np.mean(recall_scores)
    avg_mrr = np.mean(mrr_scores)

    print(f"✅ Evaluation Results:")
    print(f"🔹 Recall@{top_k}: {avg_recall:.4f}")
    print(f"🔹 MRR@{top_k}:    {avg_mrr:.4f}")

    return avg_recall, avg_mrr


In [ ]:
evaluate_cross_lingual_rag(
    eval_data=test_set,         # List[dict] with question + relevant_cids
    corpus=corpus,              # List[dict] with cid + text
    retrieve_bm25=retrieve_bm25,
    dense_func=dense_func,
    model_path=best_ckpt_path,
    model_name=model_name,
    top_k=10,
    device="cuda"  # hoặc "cpu"
)
